# Data Preprocessing Decisions
- Transaction dates were converted to a standard datetime format to enable time-based analysis and feature creation.
- Missing numerical values in transaction amounts and customer satisfaction scores were handled using median imputation to minimize the influence of extreme values and preserve overall data distribution.
- Extreme transaction amounts were capped at the 99th percentile to reduce the impact of outliers on aggregated customer spending metrics.
- Customer satisfaction scores were clipped to a maximum value of 10 to ensure consistency with the expected rating scale.
- Duplicate records were removed from all datasets to maintain data integrity, using unique identifiers such as Transaction_ID and Product_ID.
- Non-essential or redundant fields (e.g., target age group in the product dataset) were removed to simplify the dataset and focus on analytically relevant attributes.
- Datasets were aggregated and merged at the customer level to support customer-centric analysis aligned with FinMark’s business objectives.

# Feature Engineering Decisions
- Transaction data was aggregated per customer to generate key behavioral metrics, including total spend, average transaction value, transaction frequency, and number of unique transaction types.
- A recency feature was created by calculating the number of days since a customer’s most recent transaction, enabling analysis of customer engagement over time.
- The most frequent transaction type per customer was identified to capture primary customer behavior patterns.
- Customer feedback data was aggregated to compute average satisfaction and likelihood-to-recommend scores.
- For customers without feedback records, missing feedback metrics were filled using median values to retain these customers in downstream analysis.
- A High Spender indicator was created based on the top 25% of total customer spending to support segmentation and targeting use cases.
- A Loyalty Index was engineered by combining customer satisfaction and transaction frequency, reflecting both engagement and sentiment in a single metric.

# Data Export
- Cleaned datasets and the engineered customer feature dataset were exported as CSV files to support reproducibility and downstream modeling or reporting tasks.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load the datasets
transactions = pd.read_csv('Transaction_Data.csv')
products = pd.read_csv('Product_Offering_Data.csv')
feedback = pd.read_csv('Customer_Feedback_Data.csv')

In [3]:
# --- MERGING BEFORE CLEANING ---

# Merge transaction data with customer feedback at the transaction level
tx_cust = pd.merge(
    transactions,
    feedback,
    on='Customer_ID',
    how='left'
)

In [4]:
# --- PREPROCESSING ON MERGED DATA ---

# 1. Convert transaction date to datetime
tx_cust['Transaction_Date'] = pd.to_datetime(tx_cust['Transaction_Date'])

# 2. Handle missing transaction amounts using median of the merged data
tx_cust['Transaction_Amount'] = tx_cust['Transaction_Amount'].fillna(
    tx_cust['Transaction_Amount'].median()
)

# 3. Cap extreme transaction amounts at the 99th percentile
cap_val = tx_cust['Transaction_Amount'].quantile(0.99)
tx_cust['Transaction_Amount'] = tx_cust['Transaction_Amount'].clip(upper=cap_val)

# 4. Handle satisfaction score:
#    - Impute missing values with median
#    - Clip to max 10
if 'Satisfaction_Score' in tx_cust.columns:
    tx_cust['Satisfaction_Score'] = tx_cust['Satisfaction_Score'].fillna(
        tx_cust['Satisfaction_Score'].median()
    )
    tx_cust['Satisfaction_Score'] = tx_cust['Satisfaction_Score'].clip(upper=10.0)

# 5. Remove duplicate transaction records (using Transaction_ID)
tx_cust = tx_cust.drop_duplicates(subset=['Transaction_ID'])

# 6. Clean Product Offering Data separately (since it is not directly in tx_cust)
products = products.drop_duplicates(subset=['Product_ID'])
if 'Target_Age_Group' in products.columns:
    products = products.drop(columns=['Target_Age_Group'])


In [5]:
# --- FEATURE ENGINEERING (AGGREGATE PER CUSTOMER) ---

# Latest transaction date for recency computation
latest_date = tx_cust['Transaction_Date'].max()

# Aggregate transaction behavior per customer
cust_agg = tx_cust.groupby('Customer_ID').agg(
    Total_Spend=('Transaction_Amount', 'sum'),
    Avg_Transaction_Value=('Transaction_Amount', 'mean'),
    Transaction_Count=('Transaction_ID', 'count'),
    Last_Transaction_Date=('Transaction_Date', 'max'),
    Unique_Transaction_Types=('Transaction_Type', 'nunique')
).reset_index()

# Recency in days since last transaction
cust_agg['Recency'] = (latest_date - cust_agg['Last_Transaction_Date']).dt.days

# Most frequent transaction type per customer
mode_type = tx_cust.groupby('Customer_ID')['Transaction_Type'] \
                   .agg(lambda x: x.mode()[0]) \
                   .reset_index()
mode_type.rename(columns={'Transaction_Type': 'Primary_Transaction_Type'}, inplace=True)

# Aggregate feedback data at customer level
# (using the cleaned Satisfaction_Score from tx_cust)
feed_agg = tx_cust.groupby('Customer_ID').agg(
    Avg_Satisfaction_Score=('Satisfaction_Score', 'mean'),
    Avg_Likelihood_to_Recommend=('Likelihood_to_Recommend', 'mean')
).reset_index()


In [6]:
# --- MERGE INTO MASTER DATASET ---

master_df = pd.merge(cust_agg, mode_type, on='Customer_ID', how='left')
master_df = pd.merge(master_df, feed_agg, on='Customer_ID', how='left')

# Fill missing feedback metrics (customers with no feedback) using medians
master_df['Avg_Satisfaction_Score'] = master_df['Avg_Satisfaction_Score'].fillna(
    master_df['Avg_Satisfaction_Score'].median()
)
master_df['Avg_Likelihood_to_Recommend'] = master_df['Avg_Likelihood_to_Recommend'].fillna(
    master_df['Avg_Likelihood_to_Recommend'].median()
)



In [7]:
# --- NEW METRICS: HIGH SPENDER, LOYALTY INDEX, INCOME LEVEL GROUP ---

# High Spender indicator (top 25% of total spend)
master_df['Is_High_Spender'] = (
    master_df['Total_Spend'] > master_df['Total_Spend'].quantile(0.75)
).astype(int)

# Loyalty Index: satisfaction × transaction frequency
master_df['Loyalty_Index'] = (
    master_df['Avg_Satisfaction_Score'] * master_df['Transaction_Count']
)

# Income Level Group based on Total_Spend using relative positions (terciles)
# 0–33%  -> Low
# 33–66% -> Medium
# 66–100% -> High
q1 = master_df['Total_Spend'].quantile(1/3)
q2 = master_df['Total_Spend'].quantile(2/3)

def income_group(amount):
    if amount <= q1:
        return 'Low'
    elif amount <= q2:
        return 'Medium'
    else:
        return 'High'

master_df['Income_Level_Group'] = master_df['Total_Spend'].apply(income_group)


In [8]:
# --- EXPORT CLEANED AND ENGINEERED DATASETS ---

# Transaction-level merged and cleaned data
tx_cust.to_csv('Cleaned_Transactions_Merged.csv', index=False)

# Cleaned products
products.to_csv('Cleaned_Products.csv', index=False)

# Customer-level engineered features (with income level group)
master_df.to_csv('Engineered_Customer_Features.csv', index=False)

